In [26]:
from bs4 import BeautifulSoup
import requests
def scrape(course_url):
    course_page = requests.get(course_url)
    course_soup = BeautifulSoup(course_page.content, "html.parser")

    for row in course_soup.findAll('script')[4]:
        soup = BeautifulSoup(row, 'lxml')

    str_soup = soup.get_text()

    final_str = str_soup[22:len(str_soup)-94]

    final_str[final_str.find('crn":"') + 6: final_str.find('crn":"') + 11]
    
    return_string = ""
    
    course_dict = {}
    partial_str_crn = str_soup
    for row in course_soup.findAll('script')[4]:
        while ('crn' in partial_str_crn):
            temp = []
            partial_str_crn = partial_str_crn[partial_str_crn.find('crn":"') + 6:]
            partial_str_time = partial_str_crn[partial_str_crn.find('time":"') + 11:]
            partial_str_day = partial_str_crn[partial_str_crn.find('day":"') + 6:]
            partial_str_loc = partial_str_crn[partial_str_crn.find('location":"') + 11:]
            temp.append(partial_str_loc[:partial_str_loc.find('"')])
            temp.append(partial_str_time[:partial_str_time.find('"')])
            temp.append(partial_str_day[:partial_str_day.find('"')].strip())
            course_dict[partial_str_crn[:partial_str_crn.find('"')]] = temp
            #print(str(partial_str_crn[:partial_str_crn.find('"')]) + " : " + str(temp))
            return_string += str(partial_str_crn[:partial_str_crn.find('"')]) + " : " + str(temp) + '\n'

    return return_string

In [ ]:
from werkzeug.wrappers import Request, Response
from flask import Flask, render_template, request, url_for, flash, redirect
from flask_table import Table, Col

app = Flask(__name__)
app.config['SECRET_KEY'] = 'superdupersecretkeythatnobodywouldeverguess'

@app.route("/", methods=('GET', 'POST'))
def index():
    if request.method == 'POST':
        user_input = str(request.form['url'])
        return_string = scrape(user_input)
        print(return_string)
        return redirect(url_for('output', output_string = return_string))
    return render_template('index.html')

@app.route("/output/<output_string>")
def output(output_string):
    return render_template('output.html', output_string=output_string)

# Code taken from: https://stackoverflow.com/questions/41831929/debug-flask-server-inside-jupyter-notebook
if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 8080, app)

 * Running on http://localhost:8080/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Mar/2022 18:55:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2022 18:55:20] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [29/Mar/2022 18:55:20] "GET /output/69781%20:%20%5B%27329%20Gregory%20Hall%27%2C%20%2709:00AM%20-%2010:20AM%27%2C%20%27TR%27%5D%0A30107%20:%20%5B%27242%20Bevier%20Hall%27%2C%20%2709:00AM%20-%2009:50AM%27%2C%20%27F%27%5D%0A41729%20:%20%5B%27242%20Bevier%20Hall%27%2C%20%2710:00AM%20-%2010:50AM%27%2C%20%27F%27%5D%0A43832%20:%20%5B%27113%20Gregory%20Hall%27%2C%20%2711:00AM%20-%2011:50AM%27%2C%20%27F%27%5D%0A48232%20:%20%5B%27113%20Gregory%20Hall%27%2C%20%2712:00PM%20-%2012:50PM%27%2C%20%27F%27%5D%0A48234%20:%20%5B%27n.a.%27%2C%20%2701:00PM%20-%2001:50PM%27%2C%20%27F%27%5D%0A50105%20:%20%5B%27n.a.%27%2C%20%2702:00PM%20-%2002:50PM%27%2C%20%27F%27%5D%0A54572%20:%20%5B%27212%20Davenport%20Hall%27%2C%20%2710:00AM%20-%2010:50AM%27%2C%20%27F%27%5D%0A51886%20:%20%5B%27212%20Davenport%20Hall%27%2C%20%2711:0

69781 : ['329 Gregory Hall', '09:00AM - 10:20AM', 'TR']
30107 : ['242 Bevier Hall', '09:00AM - 09:50AM', 'F']
41729 : ['242 Bevier Hall', '10:00AM - 10:50AM', 'F']
43832 : ['113 Gregory Hall', '11:00AM - 11:50AM', 'F']
48232 : ['113 Gregory Hall', '12:00PM - 12:50PM', 'F']
48234 : ['n.a.', '01:00PM - 01:50PM', 'F']
50105 : ['n.a.', '02:00PM - 02:50PM', 'F']
54572 : ['212 Davenport Hall', '10:00AM - 10:50AM', 'F']
51886 : ['212 Davenport Hall', '11:00AM - 11:50AM', 'F']
70761 : ['329 Davenport Hall', '01:00PM - 01:50PM', 'F']
70762 : ['329 Davenport Hall', '02:00PM - 02:50PM', 'F']
30106 : ['n.a.', '11:00AM - 11:50AM', 'MW']

